In [29]:
!pip install numpy
!pip install pandas
!pip install nltk
import nltk
import numpy as np
import pandas as pd
import re
import string
import pickle


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
with open('../static/model/corpora/stopwords/english','r') as file:
    sw = file.read().splitlines()
    
with open('../static/model/model.pickle','rb') as f:
    model = pickle.load(f)

vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
tokens = vocab[0].tolist()

from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [14]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation,'')
    return text

In [31]:
def preprocessing(text):
    data = pd.DataFrame([text],columns=['tweet'])
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[r\n]*','',x, flags=re.MULTILINE) for x in x.split()))
    data['tweet'] = data['tweet'].apply(remove_punctuations)
    data['tweet'] = data['tweet'].str.replace('\d+','',regex=True)
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(ps.stem(x) for x in x.split() ))
    return data['tweet']

In [33]:
def vectorizer (ds, vocabulary):
    vectorized_lst = []
    for sentense in ds:
        sentense_lst = np.zeros(len(vocabulary))

        for i in range(len(vocabulary)):
            if vocabulary[i] in sentense.split():
                sentense_lst[i] = 1

        vectorized_lst.append(sentense_lst)
    vectorized_lst_new = np.asarray(vectorized_lst,dtype=np.float32)

    return vectorized_lst_new

In [39]:
def get_prediction(txt):
    preprocessed_txt = preprocessing(txt)
    vectorized_txt = vectorizer(preprocessed_txt,tokens)
    prediction = model.predict(vectorized_txt)
    if prediction == 1:
        return 'negative'
    else:
        return 'positive' 

In [48]:
get_prediction("love")

'positive'